In [4]:
import numpy as np

from ppfft.tools.pad import adj_pad
from ppfft.tools.new_fft import adj_new_fft
from ppfft.tools.frac_fft import adj_frac_fft_for_ppfft
from ppfft.ppfft.ppfft import ppfft


# Adjoint of ``ppfft``

In this notebook we define the adjoint of the PPFFT operator. Since we have already defined the adjoint operator of each function used in ``ppfft``, we only need apply them one after the other.

The following function, which defines an inner product on complex matrices, will be used to check our adjoints.


In [5]:
def adj_ppfft(hori_ppfft: np.ndarray, vert_ppfft: np.ndarray) -> np.ndarray:
    n, m = hori_ppfft.shape[0] - 1, hori_ppfft.shape[1]
    hori_aux = np.empty(shape=(n, m), dtype=complex)
    vert_aux = np.empty(shape=(n, m), dtype=complex)

    for k, (col_h, col_v) in enumerate(zip(hori_ppfft.T, vert_ppfft.T)):
        alpha = -2 * (k - n) / m
        hori_aux[:, k] = adj_frac_fft_for_ppfft(col_h, alpha)
        vert_aux[:, k] = adj_frac_fft_for_ppfft(col_v, alpha)

    hori_aux = adj_pad(adj_new_fft(hori_aux), (n, n))
    vert_aux = adj_pad(adj_new_fft(vert_aux), (n, n))

    return hori_aux + vert_aux.T

In [11]:
n = 10
a = np.random.rand(n, n)
b, c = np.random.rand(n+1, 2*n+1), np.random.rand(n+1, 2*n+1)
hori_ppfft, vert_ppfft = ppfft(a)
adj_b_c = adj_ppfft(b, c)

np.isclose(np.vdot(a, adj_b_c), (np.vdot(hori_ppfft, b) + np.vdot(vert_ppfft, c)))

True

In [12]:
n = 11
a = np.random.rand(n, n)
b, c = np.random.rand(n+1, 2*n+1), np.random.rand(n+1, 2*n+1)
hori_ppfft, vert_ppfft = ppfft(a)
adj_b_c = adj_ppfft(b, c)

np.isclose(np.vdot(a, adj_b_c), (np.vdot(hori_ppfft, b) + np.vdot(vert_ppfft, c)))

True